## 이미지 분할을 위한 신경망

- 이미지 분할(Image Segmentation) : 이미지를 픽셀 단위로 분할하여 객체를 추출하는 기술

---

#### 완전 합성곱 네트워크 FCN(Fully Convolutional Network)

- 이미지 분류 출력층에서의 FC Layer의 문제

  - **고정된 크기의 입력**만 받을 수 있음
  
  - 이를 거친 후에는 **위치 정보가 사라짐**
  
    -> `FC Layer`를 `1x1 Convolution`으로 대체하여 해결

- CNN 기반 모델(AlexNet, VGG16 등)을 변형시켜, Segmentation에 적합하도록 만듦

  - FC Layer를 1x1 Conv로 변환하면, 위치 정보가 남아있기 때문에 heatmap 그림과 같이 객체의 위치를 확인할 수 있음

<img src="https://miro.medium.com/max/875/1*ddf0sJZDitiqVCWdjijzCA.png" width="500px">

---

#### 합성곱 & 역합성곱 네트워크

- 완전 합성곱 네트워크에도 단점이 존재함

  - 여러 단계의 Conv - Pool 을 거치며 **해상도가 낮아짐**
  
  - 해상도 복원을 위한 upsampling 과정에서 이미지의 세부 정보를 잃어버림

    -> `역합성곱 네트워크`를 도입하여 해결

- 역합성곱은 일반적인 CNN과 반대로 feature map 크기를 증가시키는 방식으로 동작함

  - 각 픽셀 주위에 `zero-padding`을 추가함
  
  - padding이 적용된 것에 Convolution 연산을 수행함

<img src="https://t1.daumcdn.net/cfile/tistory/99CA8E3359FE990510" width="250px">

---

#### U-Net

##### - 속도 개선

- 기존 sliding window 방식은 이전 patch에서 체크가 끝난 부분을 다음 patch에서도 또 체크하기 때문에 속도가 느림
  
- U-Net은 이미 체크가 된 patch는 건너뛰어 속도가 빠름

<img src="https://velog.velcdn.com/images%2F00springbom00%2Fpost%2F24f86747-ef7b-41ba-a760-e723b40dfdc5%2Fimage.png" width="550px">

##### - trade-off 문제 개선 : `context 인식` vs `localization`

- patch 크기가 커진다면 넓은 범위를 인식하는데 뛰어나기 때문에, context 인식에 좋지만 localization에 한계가 있음 (반비례 관계)

  - context 인식

    - 객체가 주변 환경과 어떻게 상호작용 하는지 이해하여 객체 식별에 도움을 주는 것

    - *분류 정확도는 향상시킬 수 있지만, 객체 위치 파악은 어렵게 만들 수 있음*

  - localization

    - 객체가 이미지 내에서 어디에 위치하는지 파악하는 것
    
    - *주변 환경에 대한 정보는 상대적으로 무시될 수 있음*


##### - 구조

- 완전 합성곱 네트워크 FCN을 기반으로 구축됨

- 인코더 - 디코더 기반 모델

  - 인코더 : 수축 경로(contracting path)

    - 입력 이미지의 **context 포착**
  <br><br>
  - 디코더 : 확장 경로(expansive path)

    - upsampling 수행
    
    - 포착된 context와 결합하여 **localization** 수행

##### *일반적인 인코더 - 디코더 구조*

  - 인코딩 : 이미지의 차원을 축소해가며 **특징을 포착**함
  
  - 디코딩 : 저차원으로 인코딩된 정보를 이용하여 고차원의 이미지로 **복원**함

    -> 차원 축소 시 **객체의 위치 정보가 손실**되고, 저차원 정보를 토대로 복원 시 위치 정보 손실을 회복하기 어려움

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FOhZRr%2FbtrBLHkdIlT%2FAOXc5LT2Fk1CqTyFCZo5r1%2Fimg.png" width="350px">

##### *U-Net의 아이디어*

  - 저차원 정보 뿐만이 아니라 `고차원 정보`도 함께 이용하여, 이미지 특징 추출과 동시에 객체의 위치 파악도 하고자 함

    - 인코딩 단계의 각 레이어에서 추출한 특징을 디코딩 단계의 레이어에 합치는 방법을 사용함
    
    - 인코더 레이어와 디코더 레이어의 직접 연결 : Skip Connection

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FeJfCVN%2FbtrBMFlEca1%2F1qqTtx4f9RGNKImYclCHXk%2Fimg.png" width="350px">

- Skip Connection을 평행하게 두고 가운데를 기준으로 좌우대칭으로 레이어를 배치함

<img src="https://miro.medium.com/v2/1*lvXoKMHoPJMKpKK7keZMEA.png" width="600px">

---

##### 보간법

- 픽셀 값을 할당받지 못한 빈 공간에 값을 할당하여, 좋은 품질의 이미지/영상을 만드는 방법

- 선형 보간법 :

  - 원본 이미지의 픽셀 두 값을 사용하여, 원하는 좌표에서 새로운 픽셀 값을 계산하는 방법
<br><br>
- 양선형 보간법 :

  - 가장 가까운 픽셀 4개(A, B, C, D)에 가중치(*a*, *b*, *p*, *q*)를 곱한 값을 합하여 계산하는 방법
  
  - 픽셀당 선형 보간이 3번 수행됨

<img src="https://thebook.io/img/080289/355.jpg" width="300px">

<img src="https://thebook.io/img/080289/fn-33.jpg" width="300px">

---

##### 수용 영역

- 특정 Convolution 뉴런이 보고있는 입력 이미지의 영역

- 수용 영역에 따라 뉴런이 접근할 수 있는 context 또는 정보가 결정될 수 있음

- kernel/filter의 크기가 클수록 수용 영역이 커짐
    
    - 수용 영역이 작은 뉴런은 이미지의 작은 부분만 보며 `국소적인` 특징에 민감함

    - 수용 영역이 큰 뉴런은 이미지의 더 많은 부분을 보며 `전역적인` 특징에 민감함

- 더 큰 수용 영역을 사용하면 성능이 향상될 수 있지만, 계산량이 증가함

<img src="https://miro.medium.com/v2/resize:fit:720/format:webp/1*k97NVvlMkRXau-uItlq5Gw.png" width="500px">

- 외부 자극이 전체에 영향을 주는 것이 아니라 **특정 영역**에만 영향을 준다는 의미를 가짐

- 이미지에서 특정 위치의 픽셀들은 주변 픽셀과의 연관성은 높지만, 거리가 멀어질수록 연관성이 낮을 것임

- 따라서, 이미지 전체 영역에 대해서 동일한 중요도로 처리하는 대신에, **특정 범위**를 한정해 처리하면 더욱 효과적일 것이라는 아이디어